In [2]:
from __future__ import print_function, division
import sys
from bs4 import BeautifulSoup
import requests

requests.__path__

import pandas as pd

In [15]:
def scrape_page(Page, url):
    #url = "https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=Sturm&jahrgang=0&land_id=0&plus=1&spielerposition_id=alle&page="
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url + str(Page), headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    
    Players = soup.find_all("a", {"class": "spielprofil_tooltip"})
    Values = soup.find_all("td", {"class": "rechts hauptlink"})
    Numbers = soup.find_all("td", {"class": "zentriert"})
    Position = soup.find_all("table", {"class": "inline-table"})
    

    age = []
    club = []
    matches = []
    goals = []
    own_goals = []
    assists = []
    yellow = []
    second_yellow = []
    red = []
    sub_in = []
    sub_out = []

    #13 fields per row, 25 rows per page. 13*25 = 325
    for i in range(1, 325):
        if i % 13 == 1:
            age.append(Numbers[i].text)
        
        elif i % 13 == 3:
            
            mask = str(Numbers[i]).split('img alt="')[1]
            club.append(mask[:mask.find('"')])
        
        elif i % 13 == 4:
            matches.append(Numbers[i].text)
        elif i % 13 == 5:
            goals.append(Numbers[i].text)
        elif i % 13 == 6:
            own_goals.append(Numbers[i].text)
        elif i % 13 == 7:
            assists.append(Numbers[i].text)
        elif i % 13 == 8:
            yellow.append(Numbers[i].text)
        elif i % 13 == 9:
            second_yellow.append(Numbers[i].text)
        elif i % 13 == 10:
            red.append(Numbers[i].text)
        elif i % 13 == 11:
            sub_in.append(Numbers[i].text)
        elif i % 13 == 12:
            sub_out.append(Numbers[i].text)

    PlayersList = []
    ValuesList = []
    PositionsList = []
    Old_Values = []
    Profile_links = []
    
    for i in range(0,25):
        PlayersList.append(Players[i].text)
        ValuesList.append(Values[i].text)
        PositionsList.append(Position[i].text[len(Players[i].text):])
        
        snippet = str(Values[i]).split('title="')[1]
        Old_Values.append(snippet[:snippet.find('"')])
        
        snippet2 = str(Players[i]).split()[2].split('href="')[1][:-1]
        Profile_links.append(snippet2)

    df = pd.DataFrame({"Players":PlayersList,"Profile_Link":Profile_links,  "Club": club, "Values":ValuesList, "Old Values": Old_Values, "Position":PositionsList, "Age": age, "Matches": matches, "Goals": goals, \
                      "Own Goals": own_goals, "Assists": assists, "Yellow Cards": yellow, "Second Yellow": second_yellow, \
                      "Red Cards": red, "Sub In": sub_in, "Sub Out": sub_out})

    return(df)

In [ ]:
urls = ["https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=Sturm&jahrgang=0&land_id=0&plus=1&spielerposition_id=alle&page=", \
       "https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=Mittelfeld&jahrgang=0&land_id=0&plus=1&spielerposition_id=alle&page=", \
       "https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=Abwehr&jahrgang=0&land_id=0&plus=1&spielerposition_id=alle&page="
       ]

players = scrape_page(1, urls[0])
for i in range(3):
    for j in range(1, 21):
        players = players.append(scrape_page(j, urls[i]), ignore_index = True)
players = players.iloc[26:,:]

In [ ]:
#convert strings to numbers
for i in players.columns[6:]:
    players[i] = players[i].apply(lambda row: int(row))

In [ ]:
#convert values to millions of euros
def strip_values(i):
    if i[-1] == 'm':
        return(float(i[1:-1]))
    elif i[-1] == 'k':
        return(float(i[1:-1])/1000)

In [ ]:
#issue with last character in entries for forwards
players['Values'] = players['Values'].str[:-1]
players['Values'] = players['Values'].apply(strip_values)
players['Old Values'] = players['Old Values'].apply(strip_values)

In [ ]:
players.to_pickle("Players_Transfermarkt.pkl")